In [2]:
import numpy as np

#plotting
import matplotlib.pyplot as plt

In [3]:
#import matplotlib.style as style
#style.use("grayscale")
%matplotlib inline



#units
from astropy import units as u

#POPPY
import poppy
import sys
from poppy import fresnel


from poppy.poppy_core import _PUPIL, _IMAGE, _INTERMED

#logging
import logging
_log = logging.getLogger('poppy')
_log.setLevel(logging.DEBUG)

#make logging work in IPYTHON:
try:
    from IPython.core.debugger import Tracer; stop = Tracer()
except:
    pass

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

/Users/kuravih/anaconda/envs/opyum/lib/python2.7/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [4]:
class sinc_squared_occulter(poppy.AnalyticOpticalElement):
    """ Uniform transmission between 0 and 1.0 in intensity.

    Either a null optic (empty plane) or some perfect ND filter...
    But most commonly this is just used as a null optic placeholder """

    def __init__(self, name='occulter', **kwargs):
        poppy.AnalyticOpticalElement.__init__(self, name=name, **kwargs)

    def getPhasor(self, wave):
        y, x = wave.coordinates()
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y,x)
        amp = 1 - (np.sinc(r))**2
        return amp

In [5]:
def poppy_cor(wavefront):
    
    diam = 0.025*u.m
    fl = 0.7*u.m
    
    #Ref Figure 9
    wavefront *= poppy.CircularAperture(radius=diam.value/2.0)
    #add noise here
    wavefront.apply_lens_power(fresnel.GaussianLens(fl, name='M1'))
    wavefront *= poppy.CircularAperture(radius=diam.value/2.0)
    
    wavefront.propagate_fresnel(fl,display_intermed=False)

    wavefront *= sinc_squared_occulter();
    #wavefront *= poppy.CircularAperture(radius=250.0/pupil_ratio*prop_get_sampling(wavefront))
    
    wavefront.propagate_fresnel(fl,display_intermed=False)

    wavefront.apply_lens_power(fresnel.GaussianLens(fl, name='M2'))
    #wavefront *= poppy.CircularAperture(radius=diam/2 * 1.6)

    wavefront.propagate_fresnel(2*fl,display_intermed=False)

    wavefront.planetype = _PUPIL
    wavefront *= poppy.CircularAperture(radius=diam.value/2 * 0.53)
    
    wavefront.apply_lens_power(fresnel.GaussianLens(fl, name='Lyot stop'))

    wavefront.propagate_fresnel(fl,display_intermed=False)
    
    return wavefront

In [6]:
#Using the same simple setup used in TDEM milestone 1 report, Appendix B: Verification of PROPER (p. 26)
#Ref Figure 9 for the setup, Ref p. 27 for dimensions


lam = 500e-9

osamp=1 #oversampling factor

n=2048

#intialize wavefront
poppy_field = fresnel.FresnelWavefront(0.025*u.m, wavelength=lam, npix=n, oversample=osamp)

poppy_field = poppy_cor(poppy_field)









DEBUG:poppy:Skipping oversampling, oversample < 1 or already padded 
DEBUG:poppy:  Multiplied WF by phasor for Pupil plane: Circle, radius=0.01 m (Analytic)
DEBUG:poppy:Initialized: M1, fl =7.00e-01 m
DEBUG:poppy:------ Applying Lens: M1 ------
DEBUG:poppy:  Pre-Lens Beam Parameters: w_0:2.500e-02 m, z_w0=0.000e+00 m
z=0.000e+00 m, z_R=3.927e+03 m
DEBUG:poppy:  Beam radius at M1 =2.50e-02 m
DEBUG:poppy: input flat wavefront and M1 has output beam curvature of =-7.00e-01 m
DEBUG:poppy:M1 has a curvature of =-7.00e-01 m
DEBUG:poppy:M1 has a curved output wavefront, with waist at 0.699999977758 m
DEBUG:poppy:Post Optic Parameters:w_0:4.456e-06 m, z_w0=7.000e-01 m
z=0.000e+00 m, z_R=1.248e-04 m
DEBUG:poppy:Set output beam focal length to 0.7 m
DEBUG:poppy:Inside Rayleigh distance to Outside Rayleigh distance.
DEBUG:poppy:Applying spherical phase curvature =2.20e+07 m
DEBUG:poppy:Applying spherical lens phase =4.54e-08 1 / m
DEBUG:poppy:max_rsqd =1.25e-03 m2
DEBUG:poppy:  Multiplied WF by p